# Contour Overlay (Concept + Development notebook)

## Imports 
* _numpy_ for array math
* _astropy.io_ for reading and writing FITS cubes and images
* _matplotlib.pyplot_ for plotting spectra and images.

In [ ]:
from astropy.io import fits
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

## Read in data cube
Read in NIRSpec IFU data cube from Box.   This particular example is a simulated quasar + host galaxy.

In [ ]:
BoxPath='https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/cube_fitting/Q3D_20200407/'
fname='Q3D_NRS_491_s3d.fits'
filename=BoxPath+fname
hdul1=fits.open(filename)
hdul1.info()

with fits.open(filename, memmap=False) as hdulist:
    sci = hdulist["SCI"].data


## Sum over spectral axis

This could be any derivative data product of the same shape as the spatial dimensions of the cube (emission line flux, equivalent width, velocity map, etc...). Here we are doing one of the simplest operations - a sum, for illustrative purposes

In [ ]:
cube_sum=np.sum(sci, axis=0)

## Generate contours
User-specified contour levels in specified units. Custom colors are important, but could be added at a later stage if technically complicated.  Logarithmic and linear spacing options to auto-generate contours would be nice.

In [ ]:
min_level=np.min(cube_sum)
max_level=np.max(cube_sum)
level_colors=['blue', 'purple','red', 'magenta']
contour_levels=[max_level/10000., max_level/1000., max_level/100., max_level/10.]

## Display image and overlay contours

In [ ]:
ax = plt.subplots()[1]

#Image normalization
norm = ImageNormalize(stretch=SqrtStretch())

#Display image
image = ax.imshow(cube_sum, cmap='gray', origin='lower', norm=norm)

#Overlay contours
contour = ax.contour(cube_sum, levels=contour_levels, colors=level_colors)


# End original concept notebook
# Start development notebook

In [1]:
from jdaviz.app import Application
from glue.core import Data
app = Application(configuration='cubeviz')
# data = app.load_data(filename)
app

/Users/javerbukh/anaconda3/envs/jdaviz-dev-5/lib/python3.7/site-packages/glue/external/echo/__init__.py:3: UserWarning: glue.external.echo is deprecated, import from echo directly instead
  warnings.warn('glue.external.echo is deprecated, import from echo directly instead')
/Users/javerbukh/anaconda3/envs/jdaviz-dev-5/lib/python3.7/site-packages/setuptools/distutils_patch.py:26: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  "Distutils was imported before Setuptools. This usage is discouraged "


[]


Application(components={'g-viewer-tab': '<template>\n  <component :is="stack.container">\n    <g-viewer-tab\n …

In [2]:
app.load_data("/Users/javerbukh/Documents/manga-7495-12704-LOGCUBE_fixed.fits")

/Users/javerbukh/anaconda3/envs/jdaviz-dev-5/lib/python3.7/site-packages/glue/viewers/common/viewer.py:186: UserWarning: Add large data set?
  warnings.warn(message)
/Users/javerbukh/anaconda3/envs/jdaviz-dev-5/lib/python3.7/site-packages/glue/viewers/common/viewer.py:186: UserWarning: Add large data set?
  warnings.warn(message)
/Users/javerbukh/anaconda3/envs/jdaviz-dev-5/lib/python3.7/site-packages/glue/viewers/common/viewer.py:186: UserWarning: Add large data set?
  warnings.warn(message)
ERROR:root:Data should be 3- or 4-dimensional
ERROR:root:Data should be 3- or 4-dimensional
ERROR:root:Data should be 3- or 4-dimensional
ERROR:root:Could not determine format - use the `format=` parameter to explicitly set the format
ERROR:root:Data should be 3- or 4-dimensional
ERROR:root:Data should be 3- or 4-dimensional
ERROR:root:Data should be 3- or 4-dimensional
ERROR:root:Data should be 3- or 4-dimensional
ERROR:root:Data should be 3- or 4-dimensional
ERROR:root:Data should be 3- or 4-dim

# Test code, do not run

In [ ]:
scale_x = LinearScale(min=0, max=1)
scale_y = LinearScale(min=0, max=1)
scales = {'x': scale_x, 'y': scale_y}
axis_x = Axis(scale=scale_x, label='x')
axis_y = Axis(scale=scale_y, label='y', orientation='vertical')
scales_image = {'x': scale_x, 'y': scale_y, 'image': ColorScale(min=np.min(data_at_slice).item(), max=np.max(data_at_slice).item())}
# print(np.min(cube_sum).item(), np.max(cube_sum).item())

figure = Figure(scales=scales, axes=[axis_x, axis_y])
image = ImageGL(image=data_at_slice, scales=scales_image)
contour = Contour(contour_lines=[contour_list], level=contour_levels, scales=scales_image, label_steps=200)
#contour = Contour(image=image, level=contour_levels, scales=scales_image, color="orange")

figure.marks = (image, contour)
figure

In [ ]:
figure_flux.figure.marks = figure_flux.figure.marks + [contour]

In [ ]:
figure_flux.figure.marks = figure_flux_original_marks

Plugin idea: Have the user enter what viewer they would like the contours to appear, what they would like contoured, contour levels, whether it updates with slice changed, and visible bool.



In [ ]:
contour_level = [0.00016383392810821534, 0.0016383392810821534, 0.01638339281082153, 0.16383392810821534]
contour_level_str = ["{0:.4g}".format(level) for level in contour_level]
print(",".join(contour_level_str))

# Run starting here

In [3]:
from bqplot import Figure, LinearScale, Axis, ColorScale
from bqplot_image_gl import ImageGL, Contour
import skimage.measure
import numpy as np

In [4]:
viewer = app.get_viewer('flux-viewer')
# [layer_state.layer.label for layer_state in viewer.state.layers]
original_marks = viewer.figure.marks

# Print data from viewer
data_from_viewer = viewer.data()
print(data_from_viewer[0].get_object().unmasked_data[viewer.state.slices[0],:,:].min())

-0.014779135584831238 1e-17 erg / (A cm2 s)


In [5]:
# Get data at current slice in the app
data = app.data_collection["manga-7495-12704-LOGCUBE_fixed.fits[FLUX]"].get_object()
slice_index = viewer.state.slices[0]
data_at_slice = data.unmasked_data[slice_index, :, :].value

# Set test contour levels
contour_levels = [0.01708,0.0708,0.1417,0.4,0.7,0.9,1.1]
contour_levels_str = ["{0:.4g}".format(level) for level in contour_levels]
color_list = ["red", "orange", "yellow", "green", "blue"]
# print(data_at_slice[50])
# viewer.figure.marks[0].image.min()

In [6]:
# Testing with different scales

scale_x = LinearScale(min=0, max=1)
scale_y = LinearScale(min=0, max=1)

# Original scales
scales_image = {'x': scale_x, 'y': scale_y,
                'image': ColorScale(min=np.min(data_at_slice).item(), max=np.max(data_at_slice).item())}


# This should set the contour marks track the figure marks as they are pan and zoomed
scales = {'x': viewer.figure.interaction.x_scale, 'y': viewer.figure.interaction.y_scale}

scales_metadata = viewer.figure.marks[0].scales_metadata

In [7]:
for index in range(0,len(contour_levels)):
    contours = skimage.measure.find_contours(data_at_slice, contour_levels[index])
    
    print("Level {}\n#############################\n".format(index))
    contour = Contour(contour_lines=[contours], color=color_list[index%(len(color_list)-1)], level=float(contour_levels_str[index]), scales=scales, scales_metadata=scales_metadata, label_steps=200)

    viewer.figure.marks = viewer.figure.marks + [contour]

Level 0
#############################

Level 1
#############################

Level 2
#############################

Level 3
#############################

Level 4
#############################

Level 5
#############################

Level 6
#############################



In [ ]:
# Reset viewer marks
viewer.figure.marks = [viewer.figure.marks[0]]

In [11]:
from bqplot import Toolbar
from ipywidgets import VBox
my_toolbar = Toolbar(figure=viewer.figure)
VBox([viewer.figure, my_toolbar])

ZeroDivisionError: float division by zero

ZeroDivisionError: float division by zero

ZeroDivisionError: float division by zero